In [1]:
import torch

# 1. Scalar example: verify derivative

In [2]:
# x is a leaf tensor we care about
x = torch.tensor(2.0, requires_grad=True)
print("x:", x, "requires_grad:", x.requires_grad)

x: tensor(2., requires_grad=True) requires_grad: True


In [3]:
# Define a simple function: y = x^2 + 3x + 1
y = x**2 + 3*x + 1
print("y:", y)

y: tensor(11., grad_fn=<AddBackward0>)


In [4]:
# Backprop: dy/dx goes into x.grad
y.backward()
print("x.grad:", x.grad)  # should be 2*x + 3 = 7.0

x.grad: tensor(7.)


# 2. Vector → scalar: gradient for each element

In [5]:
x = torch.linspace(-2, 2, steps=5, requires_grad=True)
print("x:", x)

x: tensor([-2., -1.,  0.,  1.,  2.], requires_grad=True)


In [6]:
# Example: y = sum(x^2)
y = (x**2).sum()
print("y (scalar):", y)

y (scalar): tensor(10., grad_fn=<SumBackward0>)


In [7]:
y.backward()
print("x.grad:", x.grad)  # Should be 2*x for each element

x.grad: tensor([-4., -2.,  0.,  2.,  4.])


In [8]:
x.grad.zero_()

z = (3*x + 1).sum()
z.backward()
print("x.grad:", x.grad)  # Should be 3 for all elements

x.grad: tensor([3., 3., 3., 3., 3.])


# 3. Non-scalar output: use gradient argument

In [9]:
x = torch.arange(4.0, requires_grad=True)  # [0,1,2,3]
y = x**2
print("x:", x)
print("y:", y)  # non-scalar

x: tensor([0., 1., 2., 3.], requires_grad=True)
y: tensor([0., 1., 4., 9.], grad_fn=<PowBackward0>)


In [10]:
# Provide gradient (same shape as y)
grad_output = torch.ones_like(y)
y.backward(grad_output)

In [11]:
print("x.grad:", x.grad)  # Again 2*x

x.grad: tensor([0., 2., 4., 6.])


# 4. Stop tracking and detach

In [12]:
a = torch.randn(3, requires_grad=True)
b = 2 * a
c = b.mean()

In [13]:
print("a.requires_grad:", a.requires_grad)
print("b.requires_grad:", b.requires_grad)

a.requires_grad: True
b.requires_grad: True


In [14]:
c.backward()
print("a.grad:", a.grad)

a.grad: tensor([0.6667, 0.6667, 0.6667])


In [19]:
# Now detach b so it no longer tracks gradients
a.grad.zero_()
b_detached = b.detach()
d = b_detached.mean()
print("b_detached.requires_grad:", b_detached.requires_grad)

b_detached.requires_grad: False


# 5. Gradients accumulate and need zeroing

In [20]:
w = torch.tensor(2.0, requires_grad=True)

for step in range(3):
    y = (w - 1)**2
    y.backward()
    print(f"step {step}, w.grad:", w.grad)

    # Zero grad to avoid accumulation
    w.grad.zero_()

step 0, w.grad: tensor(2.)
step 1, w.grad: tensor(2.)
step 2, w.grad: tensor(2.)
